## Extracting patterns on leads 

We are going to play with the following dataset

1.- **Dataset name to request access:** Insights-FactLayer-Leads

2.- **Athena (SQLaaS):** {provider}_databox.insights_leads_fact_layer_90d.

3.- **S3 path** s3://schibsted-spt-common-prod/yellow/insights/leads/

[More information](https://docs.schibsted.io/data-and-insight/insights-pipelines/10.Data%20Model/fact-layer/#sessions-user-behaviour)

In [ ]:
# Needed packages
from pyathena import connect
import pandas as pd
import os

In [ ]:
from getpass import getpass
access_key = getpass(prompt="Enter your access key to databox: ")
secret_key = getpass(prompt="Enter your secret to databox: ")

# Some parameters (another different way to extract the credential)
user = "maria.pelaez@schibsted.com/"
provider ='avitoma'

# Doing the connection
conn = connect(aws_access_key_id=access_key,
               aws_secret_access_key=secret_key,
               s3_staging_dir="s3://schibsted-spt-common-dev/user-areas/"+ user,
               region_name="eu-west-1")

In [ ]:
# Doing a simple query
query_leads = """
SELECT
  *
FROM
  {}_databox.insights_leads_fact_layer_90d
LIMIT 10
"""
df_leads = pd.read_sql(query_leads.format(provider), conn)

In [ ]:
df_leads.count()

In [ ]:
df_leads.head()

In [ ]:
# Doing a simple query
leads = """
SELECT
 globalleadtype,
 published,
 count(globalleadtype) AS nof_leads
FROM
(
 SELECT
   globalleadtype,
   substring(published,1,10) as published
 FROM
  {}_databox.insights_leads_fact_layer_90d  
)
GROUP BY 
 globalleadtype,
 published
"""
df = pd.read_sql(leads.format(provider), conn)

In [ ]:
df.head()

In [ ]:
df['globalleadtype'].unique()

### Step 1: Prepare your data to plot

In [ ]:
tabla = df.pivot_table(index='published',columns='globalleadtype',values='nof_leads')

In [ ]:
tabla.head()

### Step 2: Create plot

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig,ax = plt.subplots(1,1,figsize=(15,10))
tabla.plot(ax=ax)

### Step 3: Customize the plot

In [ ]:
## Improving the labels and adding title
ax.set_xlabel("")
fig.suptitle('Evolution of leads per type',fontsize=20)
fig

In [ ]:
## fixing size of ticks
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)
fig

In [ ]:
# Modifying the legend
ax.legend(loc='upper left',prop={'size':18})
fig

In [ ]:
## Un barplot
import datetime as dt

In [ ]:
df['weekdate'] = pd.to_datetime(df['published']).dt.week

In [ ]:
ax = df.groupby(['weekdate'])['nof_leads'].sum().plot(kind='bar')

## **Exercise 1**:

Customize the previous graph

## **Exercise 2**:

* Create a query to extract number of lead types (sms, show phone etc.) grouped by vertical using `adlocalvertical` column
* Create a chart to visualize it.

## **Exercise 3**:

Now we are going to create two datasets: 

* All items with the number of classified ads from {provider}_databox.yellow_pulse_simple_1d (type = "View", objecttype = "ClassifiedAd")
* All items with the number of leads
    


In [ ]:
query_data1 = """
SELECT
 objectid,
 category,
 count(*) AS nof_adviews
FROM
 {}_databox.yellow_pulse_simple_1d
WHERE 
 (type='View' and objecttype = 'ClassifiedAd')
GROUP BY
 objectid,
 category
"""


query_data2 = """
SELECT
  objectid,
  count(*) as_nof_leads
FROM
 {}_databox.yellow_pulse_simple_1d
WHERE 
 (type='Call' and objecttype = 'PhoneContact') OR
 (type='Show' and objecttype = 'PhoneContact') OR
 (type='Send' and objecttype = 'Message') OR
 (type='SMS' and objecttype = 'PhoneContact')
GROUP BY
 objectid
"""
adviews = pd.read_sql(query_data1.format(provider), conn)
leads = pd.read_sql(query_data2.format(provider), conn)

Compute:
    
1) A new dataframe joining adviews and leads

2) To compute the percentage of ads with leads per category

3) How many ad views are needed to have at least one lead per category?


**Hint**: Check [this](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) out to learn how to merge dataframes.